<a href="https://colab.research.google.com/github/seunghee0518/R_File/blob/main/R_%EB%8B%A4%EB%B3%80%EB%9F%89%EB%B6%84%EC%84%9D_7%EC%9E%A5_%ED%8C%90%EB%B3%84%EB%B6%84%EC%84%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#예제

##2. 분류계수함수를 이용한 분류방법
> 분류함수 계수를 구하는 function문

In [ ]:
#Function to calculate Fisher`s Classification coefficients
#this code follows the formulas in Legendare and Legendre`s Numerical Ecology(1998)
classfunc.lda <- function(x, groups){
  x.lda <- lda(groups ~ ., as.data.frame(x))

  gr <- length(unique(groups))  #groups might be factors or numeric
  v <- ncol(x)                  #variables
  m <- x.lda$means              #group means

  w <- array(NA, dim = c(v, v, gr))

  tot.len <- nrow(x)
  subgr.len <- array(0, gr)

  for(i in 1:gr){
    tmp <- scale(subset(x, groups = unique(groups)[i]), scale=FALSE)
    w[,,i] <- t(tmp) %*% tmp
    subgr.len[i] <- nrow(tmp)
  }

  W <- w[,,1]
  for(i in 2:gr){
    W <- W + w[,,i]
  }

  V <- W/(nrow(x) - gr)
  iV <- solve(V)

  class.func <- matrix(NA, nrow= v + 1, ncol = gr)

  colnames(class.func) <- rownames(m)
  rownames(class.func) <- c("constant", colnames(m) )

  for(i in 1:gr) {
    ni <- subgr.len[i]
    class.func[1,i] <- -0.5*t(m[i,]) %*% iV %*% (m[i,]) + log(ni / tot.len)
    class.func[2:(v+1) ,i] <- iV %*% (m[i,])   
  } 

  x.lda$class.func <- class.func
  return(x.lda)
}

#위 프로그램을 실행하기 위한 코드
#source("c:/data/mva/classfunc.r")

In [ ]:
#만든 코드로 실행하기 => 분류함수 계수 알려줌
library(MASS)
data_al = read.csv("https://raw.githubusercontent.com/seunghee0518/R_File/main/%EB%8B%A4%EB%B3%80%EB%9E%91%EB%B6%84%EC%84%9D/mvadata/alcohol.csv", header=T)
X = data_al[, -1]
classfunc.result = classfunc.lda(X, data_al$TYPE)
names(classfunc.result)
classfunc.result$class.func

##3. 사후확률이용방법+판별모형 평가(분류표) (p.200)

In [ ]:
#MASS는 기본설치패키지, 따로 install필요 X
library(MASS)
data7 = read.table("https://raw.githubusercontent.com/seunghee0518/R_File/main/%EB%8B%A4%EB%B3%80%EB%9E%91%EB%B6%84%EC%84%9D/mvadata/data7-1.txt", header=T)
head(data7)

In [ ]:
data7.lda = lda(group ~ ., data=data7)
pred.lda = predict(data7.lda, newdata = data7)
pred.lda$class

In [ ]:
pred.lda$posterior

##예제 (사후확률법 이용)
>
알콜 종류 3가지, 독립변수 6개, 케이스 수가 77개인 자료임, MASS패키지의 lda()함수를 이용해 판별분석하기

In [ ]:
#데이터 읽기
url_a = "https://raw.githubusercontent.com/seunghee0518/R_File/main/%EB%8B%A4%EB%B3%80%EB%9E%91%EB%B6%84%EC%84%9D/mvadata/alcohol.csv"
alcohol = read.csv(url_a, header=T)
attach(alcohol)
head(alcohol, 3)

In [ ]:
#기초통계량
summary(alcohol)

> lda(그룹변수 ~ 나머지변수, data = )
>
 prior probabilities or groups = 각 그룹별 사전확률값
 그룹수가 3개이므로 2개의 선형판별함수

In [ ]:
#판별분석 실행
library(MASS)
alcohol.lda = lda(TYPE~., data = alcohol)
alcohol.lda

In [ ]:
#분류하기(사후확률법으로 분류됨)
pred.lda = predict(alcohol.lda, newdata=alcohol)

In [ ]:
#pred.lda의 속성변수 확인
names(pred.lda)

In [ ]:
#맨처음6개 케이스-> 사후확률값이 큰 그룹으로 분류됨
head(pred.lda$class)

In [ ]:
#사후확률값 조회
tail(pred.lda$posterior)

In [ ]:
#각 케이스의 판별점수
head(pred.lda$x)

In [ ]:
#분류표
confm.lda = table(alcohol$TYPE, pred.lda$class)
confm.lda

In [ ]:
#오분류율
accuracy = sum(diag(confm.lda))/sum(confm.lda)
accuracy
error = 1-accuracy
error

In [ ]:
#판별변수 선택-패키지 설치
install.packages("klaR")
library(klaR)

In [ ]:
#판별변수 선택
alcohol.forward = greedy.wilks(TYPE ~ ., data=alcohol, niveau=0.01)
alcohol.forward

In [ ]:
#변수 선택 후 판별분석 실행
alcohol.fwd.lda = lda(alcohol.forward$formula, data=alcohol)
alcohol.fwd.lda

In [ ]:
#변수선택 후 분류표
pred.fwd.lda = predict(alcohol.fwd.lda, newdata=alcohol)
confm.fwd = table(alcohol$TYPE, pred.fwd.lda$class)
confm.fwd

#실습

##1. 어느 시점을 기준으로 도산한 기업과 정상운영 기업을 대상으로 지난 2년 동안에 발생된 재무상황의 변동을 조사한 결과이다. 각 변수의 의미는 다음과 같은데 해당 자료로 판별분석과 그 결과를 해석하라
>
X1 : 운영자금/총 자산
X2 : 판매수익/총 자산
X3 : 세전총소득/총 자산
X4 : 시장가치자산/총 자산
X5 : 판매액/총 자산

##2. 공업도시와 상업도시별로 인구수(단위:만 명)와 대기오엽도를 조사한 자료, 판별분석과 결과 해석

##3. 어느 한 시점에서 도산한 기업과 도산하지 않은 기업들의 2년 전 재무현황 자료, 이 자료 중 앞부분을 이용하여 판별분석을 수행한 후, 이를 이용하여 나머지 기업에 대한 2년 후 기업들의 도산 여부를 분류하라
>
X1 : 총 부채에 대한 유동성 비율
X2 : 총 자산에 대한 순이익 비율
X3 : 총 부채에 대한 현 자산 비율
X4 : 순판매에 대한 현 자산 비율
1. 판별분석하기
2. 2년 후 자료를 분류하기